In [ ]:
import os
import tiktoken
import difflib
import pandas as pd
import io
import sys
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
from langchain_openai import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import requests
import urllib.parse


In [ ]:
llm_4omini = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint_4mini'), 
    openai_api_version="2024-10-21",   
    model_name="gpt-4o-mini",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key_4mini'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt4o-mini")
    #streaming=True

In [ ]:
fr_old_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [ ]:
fr_new_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, d'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [ ]:
def get_eu_data_3p(directive_UE_CRR_step4, fr_old_version, fr_new_version):
    prompt_LCQA_3p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise avec pour objectif de trouver la ou les parties
        de la directive UE (article ou articles) a l origine de ce(s) changement(s).
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        - Tu ne pas pas reciter le contenu de la loi francaise avant et après modification
        
        Voici le text européen {document_aml_pdf_step_4}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
"""
    for chunk in llm_4o.stream(prompt_LCQA_3p):
        print(chunk.content, end="", flush=True)

In [ ]:
get_eu_data_3p(fr_old_version,fr_new_version)